In [1]:
#python specific stuff
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from functions import *
from ReadInParameters import *

##TODO Lizzy
#variable time step? check convergence somehow? Runge kutta method
#optimize run time
#internal volumes
#different types of windows
#initial = external temp
#show progress bar
#internals - end cell no heat loss
#modulate radiant on/off
#underfloor

#FOR DAD
# Check material values
# how to do windows double glazing
# make Qin model 


In [2]:
## INITIALIZATION for time, and distance step?
total_time_h=3 #hours
t_step=0.1 #s
total_time_s=total_time_h*60*60
N_t_steps=int(total_time_s/t_step)
cell_length=0.5

In [3]:
# # UPPER ROOM INITIALIZATION #########################
[lower,middle,upper,output_surface_array_lower,output_surface_array_middle,output_surface_array_upper,T_target,T_exterior,target_level]=ReadInParameters('UserInputSheet.xlsx')

                          Unnamed: 2 Unnamed: 3
7                               WALL        NaN
8                         area (m^2)        225
9             internal material type      Brick
10  internal material thickness (cm)       17.5
11            external material type      Brick
12  external material thickness (cm)          0
13               Radiant Heat in (W)          0
                         Unnamed: 4     Unnamed: 5
0                          Hot Spot            NaN
1                        area (m^2)              5
2            internal material type  Stramit board
3  internal material thickness (cm)             10
4            external material type          Slate
5  external material thickness (cm)              0
6               Radiant Heat in (W)              0


C:\Users\EH842\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
## DO CALCULATION ##################

data_array_T_room_upper=np.zeros(int(N_t_steps))
data_array_T_room_middle=np.zeros(int(N_t_steps))
data_array_T_room_lower=np.zeros(int(N_t_steps))
Qs=np.zeros(int(N_t_steps))
data_array_surfaces=np.zeros([int(N_t_steps),10])
Qin_cumul=[0]

for t in tqdm(range(N_t_steps)):
    
    #data input for plotting

    data_array_T_room_upper[t]=upper.T_room
    data_array_T_room_middle[t]=middle.T_room
    data_array_T_room_lower[t]=lower.T_room
    data_array=[]
    for output_surface in output_surface_array_upper:
            data_array=np.append(data_array,output_surface.T_array[0])
    for output_surface in output_surface_array_middle:
            data_array=np.append(data_array,output_surface.T_array[0])
    for output_surface in output_surface_array_lower:
            data_array=np.append(data_array,output_surface.T_array[0])
            
    data_array_surfaces[t,:]=data_array
    
  #  print(output_surface_array_upper[2].T_array[t])
    # #temp monitoring
    Qin_total=0
    if lower.T_room >= T_target:
        upper.Qin=0
        middle.Qin=0
        lower.Qin=0
        for surface in output_surface_array_upper+output_surface_array_middle+output_surface_array_lower:
            surface.Qrad=0
    else:
        upper.Qin=upper.Qin_max
        middle.Qin=middle.Qin_max
        lower.Qin=lower.Qin_max
                Qin_total=Qin_total+upper.Qin+middle.Qin+lower.Qin
        for surface in output_surface_array_upper+output_surface_array_middle+output_surface_array_lower:
            surface.Qrad=surface.Qrad_max
            Qin_total=Qin_total+surface.Qrad
    
    ## LAYER 1
    #do surfaces in upper room
    [Qout_room_total,output_surface_array_upper]=do_surfaces_in_room(output_surface_array_upper,upper.T_room,t_step,T_exterior)
    #now conductive to lower room
    Qout_upper_to_middle=calculate_Qout_convective(upper.U,upper.area,upper.T_room,middle.T_room)
    Qout_room_total=Qout_room_total+Qout_upper_to_middle

    #update Temp of room using Qins and Qouts
    Q_leak_upper=upper.airchanges_per_hour*upper.volume*t_step*upper.densityC*(upper.T_room-T_exterior)/3600
    upper.T_room=upper.T_room+calculate_Tchange(upper.Qin-Q_leak_upper,Qout_room_total,t_step,upper.densityC,upper.volume)
    
    
    ## LAYER 2
    [Qout_room_total,output_surface_array_middle]=do_surfaces_in_room(output_surface_array_middle,middle.T_room,t_step,T_exterior)
    Qout_middle_to_lower=calculate_Qout_convective(middle.U,middle.area,middle.T_room,lower.T_room)
    Qout_room_total=Qout_room_total+Qout_middle_to_lower
    
    #Calculate cumulative heat input:
    Q_leak_middle=middle.airchanges_per_hour*middle.volume*t_step*middle.densityC*(middle.T_room-T_exterior)/3600
    middle.T_room=middle.T_room+calculate_Tchange(middle.Qin+Qout_upper_to_middle-Q_leak_middle,Qout_room_total,t_step,middle.densityC,middle.volume)
    
    ## LAYER 3
    [Qout_room_total,output_surface_array_lower]=do_surfaces_in_room(output_surface_array_lower,lower.T_room,t_step,T_exterior)
    Q_leak_lower=lower.airchanges_per_hour*lower.volume*t_step*lower.densityC*(lower.T_room-T_exterior)/3600
    lower.T_room=lower.T_room+calculate_Tchange(lower.Qin+Qout_middle_to_lower-Q_leak_lower,Qout_room_total,t_step,lower.densityC,lower.volume)
    
    ## CUMUL Q
    Qin_cumul=np.append(Qin_cumul,Qin_cumul[t]+Qin_total*t_step)  

 95%|█████████▍| 102472/108000 [04:08<00:16, 337.17it/s]

In [ ]:
# MAKE PLOTS ##########################

t_array=np.arange(0,N_t_steps*t_step,t_step)/(60*60)
plt.plot(t_array,data_array_T_room_upper,label='T room_upper')
plt.plot(t_array,data_array_T_room_middle,label='T room_middle')
plt.plot(t_array,data_array_T_room_lower,label='T room_lower')
plt.plot(t_array,np.full(N_t_steps,T_target),label='T target')
plt.plot(t_array,np.full(N_t_steps,T_exterior),label='T exterior')
plt.legend()
plt.grid('True')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature (C)')
print(data_array_T_room_lower[-1])

In [ ]:
lineObjects =plt.plot(t_array,data_array_surfaces[:,0:4])
plt.legend(iter(lineObjects), ('Roof','Wall','Windows','Hot Spot'))
plt.title('Upper')


In [ ]:
lineObjects =plt.plot(t_array,data_array_surfaces[:,4:8])
plt.legend(iter(lineObjects), ('Roof','Wall','Windows','Hot Spot'))
plt.title('Middle')

In [ ]:
lineObjects =plt.plot(t_array,data_array_surfaces[:,8:13])
plt.legend(iter(lineObjects), ('Roof','Wall','Floor','Windows','Hot Spot'))
plt.title('Lower')

In [ ]:
plot(Qin_cumul)